# SysArmy - Análisis de encuesta 2020.02 - COVID

## 1. Carga y sanitización de datos

###  1.1. Definimos los metadatos asociados al dataset que vamos a analizar

In [1]:
%load_ext nb_black

csv_url = "https://raw.githubusercontent.com/openqube/openqube-sueldos/master/data/csv/argentina/2020.02.csv"

dataset_metadata = {
    "year": 2020,
    "part": 2,
    "skiprows": 0,
    "publish_date": "2020-08-15",
    "minimum_salary": 16875,  # https://es.wikipedia.org/wiki/Anexo:Salario_m%C3%ADnimo_en_Argentina
}

<IPython.core.display.Javascript object>

### 1.2. En base a los metadatos, cargamos el CSV, limpiamos columnas y las transformamos

In [2]:
import pandas as pd
import io
import requests
from time import sleep
import matplotlib.pyplot as plt

raw_dataframe = None
source = requests.get(csv_url).content
raw_dataframe = pd.read_csv(
    io.StringIO(source.decode("utf-8")),
    skiprows=dataset_metadata["skiprows"],
    thousands=".",
    decimal=",",
)

import re

regex_white_chars = re.compile(r"\s+")
regex_non_alphanum = re.compile(r"\W+_?", re.I)

raw_dataframe.columns = (
    raw_dataframe.columns.str.strip()
    .str.lower()
    .str.replace(regex_white_chars, "_")
    .str.replace(regex_non_alphanum, "")
    .str.translate(
        "ñáéíóú".maketrans({"ñ": "n", "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u"})
    )
)

# Eliminamos los datos que no pertenezcan a Argentina
raw_dataframe = raw_dataframe[raw_dataframe["estoy_trabajando_en"] == "Argentina"]

# Conservamos únicamente las columnas relacionadas a COVID
columns_to_keep = [
    "timestamp",
    "me_identifico",
    "donde_estas_trabajando",
    "trabajo_de",
    "tipo_de_contrato",
    "como_venis_llevando_la_cuarentena",
    "tenes_hijosas_menores_de_edad",
    "con_cuantas_personas_estas_conviviendo",
    "con_quienes_convivis",
    "tenes_un_espacio_dedicado_para_el_trabajo",
    "tenes_que_compartir_tu_equipo_de_trabajo_con_alguien",
    "que_tipo_de_cuarentena_estas_haciendo",
    "cambio_tu_situacion_laboral_a_raiz_de_la_pandemia",
    "que_tanto_sentis_que_te_esta_apoyando_tu_empresa_en_esta_situacion",
    "como_se_vio_afectada_tu_empresa",
    "instauraron_algun_beneficio_nuevo",
]

raw_dataframe = raw_dataframe[columns_to_keep]


df = raw_dataframe.copy()
df.head()

,timestamp,me_identifico,donde_estas_trabajando,trabajo_de,tipo_de_contrato,como_venis_llevando_la_cuarentena,tenes_hijosas_menores_de_edad,con_cuantas_personas_estas_conviviendo,con_quienes_convivis,tenes_un_espacio_dedicado_para_el_trabajo,tenes_que_compartir_tu_equipo_de_trabajo_con_alguien,que_tipo_de_cuarentena_estas_haciendo,cambio_tu_situacion_laboral_a_raiz_de_la_pandemia,que_tanto_sentis_que_te_esta_apoyando_tu_empresa_en_esta_situacion,como_se_vio_afectada_tu_empresa,instauraron_algun_beneficio_nuevo
0,7/6/2020 9:56:02,Mujer,Ciudad Autónoma de Buenos Aires,Consultant,Full-Time,3,0,0,Vivo solo/a,No,No,4,Decidí buscar otro trabajo,7,No la afectó,No
1,7/6/2020 9:57:09,Hombre,Corrientes,Developer,Full-Time,2,0,1,-,No,No,3,"Cambié de trabajo, pero no por la pandemia",8,No la afectó,No
2,7/6/2020 9:58:09,Mujer,Ciudad Autónoma de Buenos Aires,Infosec,Full-Time,3,0,1,Pareja,No,No,4,"Cambié de trabajo, pero no por la pandemia",8,Se vio beneficiada,"Clases de gimnasia, Clases de meditación, Clas..."
3,7/6/2020 9:59:52,Hombre,Corrientes,SysAdmin / DevOps / SRE,Full-Time,2,0,1,Pareja,Sí,No,4,No,4,"Cerró oficinas, Bajaron los sueldos o pagan pa...",No
4,7/6/2020 10:01:27,Hombre,Ciudad Autónoma de Buenos Aires,Developer,Full-Time,2,1,2,"Pareja, Hijos/as (tiempo completo)",Sí,No,3,No,6,Nos obligaron a tomarnos vacaciones,No


<IPython.core.display.Javascript object>